In [26]:
import pandas_datareader as pdr
import requests
import pandas as pd
from datetime import datetime, timedelta
import sqlite3 
import sql_tools
import time
import numpy as np
from IPython.display import clear_output
pick = sql_tools.IcePick(sqlite3, 'trade_history.sqlite')
con = sqlite3.connect('trade_history.sqlite')


In [145]:
def create_updates(tickers):
    updates = pd.DataFrame(index = set(tickers))
    updates['Last Updated'] = datetime(2018,1,24)
    updates.index.name = 'Ticker'
    updates.to_sql('symbol_info', con, if_exists='replace')


def add_to_history(ticker, data):
    try:
        data.to_sql(ticker, con,if_exists= 'fail')
    except ValueError:
        latest_date = pick.get_data('AMD', ['Date']).max().loc[0]
        downloaded_earliest = data['Date'].min()
        for days in 
    return 

def load_from_history(ticker, columns = ['*']):
    #if 'Date' not in columns:
    #    columns.append('Date')
    data = pick.get_data(ticker, columns).set_index('Date')
    data.index = pd.to_datetime(data.index)
    return data

def get_history_list():
    data = pick.read_sql('select ticker from symbol_info')['Ticker']
    return data

def is_in_history(ticker):
    if ticker in history_list:
        return True
    return False

history_list = get_history_list()

In [3]:
def get_tickers():
    nyse = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download')
    nasdaq = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download')
    data = nyse.append(nasdaq)
    data['Symbol']= data['Symbol'].str.replace(' ', '')
    data = data.set_index('Symbol')
    data = data[~data.index.duplicated()]
    data = data[~data.index.str.contains('\^')]
    data = data[~data.index.str.contains('\.')]
    data = data[~data.index.str.contains('\~')]
    data = data[data['MarketCap'] > 0]
    del data['Unnamed: 9']
    return data
tickers = get_tickers()

In [4]:
def exponential_backoff(ticker, *args):
    i = 0
    while True:
        try:
            print('Trying to Download: {}'.format(ticker))
            data = pdr.DataReader(ticker, 'yahoo', *args)
            return data
        except pdr.base.RemoteDataError as e:
            i = i + 1
            print(e)
            print('Error {} : Exponential Backoff time: {}'.format(i , i**2))
            
            if i > 7:
                print('Error Got too big, raising it')
                raise e 
            time.sleep(i ** 2)
def download_symbol(ticker):
    ticker = ticker.upper()
    print('Request is for {}'.format(ticker))
    
    if is_in_history(ticker):
        print('Ticker In Current Data.')
        data = load_from_history(ticker)
        index = data[~data.index.isnull()].index
        max_date = index.max()
        print('Max Date of data: {}'.format(max_date))
        print('Today: {}'.format(pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))))
        
        if max_date == pd.to_datetime(datetime.today().strftime('%Y-%m-%d')):
            print('Data already updated - Done')
            return data
        print('Data outdated: Downloading new data')
        new_data = exponential_backoff(ticker, max_date)
        new_data.columns = new_data.columns.str.replace(' ', '_')
        data = data.loc[data.index != max_date]
        print(data)
        print(new_data)
        data = data.append(new_data)
        
        data.columns = data.columns.str.replace(' ', '_')
        add_to_history(ticker, data)
        return data
    
    if not is_in_history(ticker):
        print('Ticker "{}" not in Current Data, Downloading Fresh'.format(ticker))
        data = exponential_backoff(ticker)
        print('Downloaded!')
        data.columns = data.columns.str.replace(' ', '_')
        add_to_history(ticker, data)
        return data

In [144]:
def progress_bar(i, left):
        power_of_one_dot = 100/left
        pos = int(power_of_one_dot * i)
        bar = ''
        for num in range(0, pos -1 ):
            if num %2 == 0:
                bar = bar + 'o'
            if num %2 != 0:
                bar = bar + 'O'             
        bar = bar + '>'
        for num in range(0, 100 - pos):
            bar = bar + ' '
        bar = bar + '| {:.2f} % \nDone: {} Remaining: {}'.format(100 * i/left, i, left - i )
        print(bar) 
        
        
def download_everything(ticker_list):

    temp  = pd.DataFrame(index = ticker_list)
    saved = pick.read_sql('SELECT * from symbol_info').set_index('Ticker')
    saved['Last Updated'] = pd.to_datetime(saved['Last Updated'])

    todo = []
    for index in temp.index:
        if index not in saved.index:
            todo.append(index)
    remaining = len(todo)
    
    print('Downloading Fresh')
    for i,ticker in enumerate(todo):
        progress_bar(i, remaining)
        download_symbol(ticker)
        clear_output(wait = True)

    print('Downloading Updates')
    today = datetime.today()
        
    
download_everything(tickers.index)

In [106]:
saved = pick.read_sql('SELECT * from symbol_info').set_index('Ticker')
saved['Last Updated'] = pd.to_datetime(saved['Last Updated'])
saved

,Last Updated
Ticker,
KEX,2018-01-24
DXPS,2018-01-24
AERI,2018-01-24
FL,2018-01-24
ODC,2018-01-24
KOSS,2018-01-24
NAC,2018-01-24
BRKR,2018-01-24
LSTR,2018-01-24


In [13]:
to_fix = []

In [73]:

data = pd.DataFrame(columns = tickers.index)
to_do = len(data.columns)
for i, column in enumerate(data.columns):
    progress_bar(i,to_do )
    try: 
        data[column] = load_from_history(column, ['Date', 'Close'])['Close']
    except ValueError as e:
        to_fix.append(column)
        pass
    clear_output(wait = True)
raw_data = data.copy()

oOoOo>                                                                                              | 6.77 % 
Done: 367 Remaining: 5050


KeyboardInterrupt: 

In [110]:
saved.index

Index(['KEX', 'DXPS', 'AERI', 'FL', 'ODC', 'KOSS', 'NAC', 'BRKR', 'LSTR',
       'CINR',
       ...
       'HFGIC', 'ECOL', 'CDE', 'HASI', 'AZN', 'PRCP', 'PHI', 'JUNO', 'AEIS',
       'PSCF'],
      dtype='object', name='Ticker', length=5417)

In [11]:
for column in to_fix:
    x = load_from_history(column)
    x = x[~x.index.duplicated()]
    x.to_sql(column, con, if_exists= 'replace')

In [49]:
data = raw_data.copy()
to_do = len(data.columns)
for i, col in enumerate(data.columns):
    data[col] = data[col].apply(np.log)
    progress_bar(i, to_do)
    clear_output(wait = True)

AttributeError: 'float' object has no attribute 'log'